# Comparisons for ALFA_PLUS

In [50]:
# After running find . > $file on both archives (BSC and UPCnet)
upcnet = open('/tmp/xnat_stats/ALFA_PLUS_upcnet.txt').readlines()
bsc = open('/tmp/xnat_stats/ALFA_PLUS_bsc.txt').readlines()

In [51]:
from glob import glob
import pandas as pd
import os.path as osp
upc_dcm = [(e.split('/')[-3], osp.basename(e)[:-1]) for e in upcnet if e.endswith('.dcm\n')]
bsc_dcm = [(e.split('/')[-3], osp.basename(e)[:-1]) for e in bsc if e.endswith('.dcm\n')]
print upc_dcm[:10], bsc_dcm[:10]

[('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-325-jmuk6y.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-277-ko5ex4.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-104-k4uoiq.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-53-iqcs13.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-232-jm6xo9.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-183-i87495.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-291-k88n38.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-149-ircjat.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-216-hdzpg9.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.2668.2017011014071866000-1001-348-irxlw2.dcm')] [('800', '1.2.840.113619.6.353.205296510108802724873751358051835081611-800-39-sktdig.dcm'), ('800', '1.2.840.113619.6.353.2052965101088027248737513580

Numbers of existing DICOM files on both sides:

In [52]:
print 'upc:', len(upc_dcm), 'bsc:', len(bsc_dcm)

upc: 1078174 bsc: 577868


Numbers of non-dicom items:

In [53]:
nondcm_upcnet = [e for e in upcnet if not '.dcm' in e]
nondcm_bsc = [e for e in bsc if not '.dcm' in e]
print 'upc:', len(nondcm_upcnet), 'bsc:', len(nondcm_bsc)

upc: 4428 bsc: 26809


Dictionaries by sequences:

In [54]:
# Creating dictionaries
upc_dict = {}
bsc_dict = {}

for seq, dcmfile in upc_dcm:
    upc_dict.setdefault(seq, []).append(dcmfile)
for seq, dcmfile in bsc_dcm:
    bsc_dict.setdefault(seq, []).append(dcmfile)

Example of differences for a given sequence:

In [55]:
list(set(upc_dict['601']).difference(set(bsc_dict['601'])))[:10]

['1.3.46.670589.11.78036.5.0.852.2017020715042705006-601-2143-105zt8g.dcm',
 '1.3.46.670589.11.78036.5.0.8548.2016112916425770016-601-11562-17h023k.dcm',
 '1.3.46.670589.11.78036.5.0.37960.2016111517004153010-601-1000-1hjt474.dcm',
 '1.3.46.670589.11.78036.5.0.2152.2016121308275965002-601-9553-1d9qb9v.dcm',
 '1.3.46.670589.11.78036.5.0.852.2017020713385069004-601-7325-poi8it.dcm',
 '1.3.46.670589.11.78036.5.0.3228.2016110814004749000-601-11431-pd9bn2.dcm',
 '1.3.46.670589.11.78036.5.0.6696.2016102515093008002-601-2402-1iu9ync.dcm',
 '1.3.46.670589.11.78036.5.0.1196.2016110215052576002-601-13035-1bn5ld9.dcm',
 '1.3.46.670589.11.78036.5.0.11888.2016111414292342002-601-4124-8wque.dcm',
 '1.3.46.670589.11.78036.5.0.9964.2017011114261267002-601-6701-1t50tx4.dcm']

Matching files between servers for every sequence:

In [56]:
results = []
print set(upc_dict.keys()).difference(set(bsc_dict.keys())), 'are in UPC, missing in BSC'
print set(bsc_dict.keys()).difference(set(upc_dict.keys())), 'are in BSC, missing in UPC'
common_sq = set(bsc_dict.keys()).intersection(set(upc_dict.keys()))
print len(common_sq), 'in common'

for each in common_sq:
    row = []
    row.append(len(upc_dict[each]))
    row.append(len(bsc_dict[each]))
    row.append(len(set(upc_dict[each]).difference(set(bsc_dict[each]))))
    row.append(len(upc_dict[each]) - len(bsc_dict[each]))
    results.append(row)
pd.DataFrame(results, index=common_sq, columns=['UPCnet', 'BSC', 'Difference after matching', 'UPCnet - BSC (raw difference)'])

set(['1304', '1301', '603', '1303', '701', '702', '1901', '1503', '0', '405', '404', '403', '1504', '1403', '1402', '1401', '501', '201', '203', '202', '1701', '102', '103', '101', '1501', '104', '903', '1801', '904', '905', '1601', '1103', '1404', '1003']) are in UPC, missing in BSC
set(['600', '700', '1200', '1', '3', '2', '5', '4', '7', '400', '9', '8', '300', '6', '305', '13', '11', '10', '900', '12', '1100', '802', '800', '1000']) are in BSC, missing in UPC
17 in common


,UPCnet,BSC,Difference after matching,UPCnet - BSC (raw difference)
902,2593,1960,2593,633
1001,31311,616,31311,30695
901,9220,1960,9220,7260
601,676349,404,676349,675945
602,3,568,3,-565
301,18289,8512,18289,9777
302,994,12800,994,-11806
303,890,256,890,634
304,90,174,90,-84
1202,1,56,1,-55


# Comparisons for ALFA1 / ALFA_OPCIONAL

In [57]:
# After running find . > $file on both archives (BSC and UPCnet)
upcnet = open('/tmp/xnat_stats/ALFA_OPCIONAL_upcnet.txt').readlines()
bsc = open('/tmp/xnat_stats/ALFA_OPCIONAL_bsc.txt').readlines()

In [58]:
from glob import glob
import pandas as pd
import os.path as osp
upc_dcm = [(e.split('/')[-3], osp.basename(e)[:-1]) for e in upcnet if e.endswith('.dcm\n')]
bsc_dcm = [(e.split('/')[-3], osp.basename(e)[:-1]) for e in bsc if e.endswith('.dcm\n')]
print upc_dcm[:10], bsc_dcm[:10]

[('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-250-1vwkjka.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-324-7oqudt.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-47-1vx5m5h.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-120-1wbg9mp.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-213-7qutva.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-169-8anfwi.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-306-8oi4pc.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-290-1vezqxr.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-71-1vex6yz.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.8760.2017012015122897000-1001-182-891yjy.dcm')] [('1001', '1.3.46.670589.11.78036.5.0.6988.2016071310315383002-1001-1-1ldwrfd.dcm'), ('1001', '1.3.46.670589.11.78036.5.0.6988.2016071310315383002

Numbers of existing DICOM files on both sides:

In [59]:
print 'upc:', len(upc_dcm), 'bsc:', len(bsc_dcm)

upc: 7286318 bsc: 7200963


Numbers of non-dicom items:

In [60]:
nondcm_upcnet = [e for e in upcnet if not '.dcm' in e]
nondcm_bsc = [e for e in bsc if not '.dcm' in e]
print 'upc:', len(nondcm_upcnet), 'bsc:', len(nondcm_bsc)

upc: 35958 bsc: 50046


Dictionaries by sequences:

In [61]:
# Creating dictionaries
upc_dict = {}
bsc_dict = {}

for seq, dcmfile in upc_dcm:
    upc_dict.setdefault(seq, []).append(dcmfile)
for seq, dcmfile in bsc_dcm:
    bsc_dict.setdefault(seq, []).append(dcmfile)

Example of differences for a given sequence:

In [62]:
list(set(upc_dict['601']).difference(set(bsc_dict['601'])))[:10]

['1.3.46.670589.11.78036.5.0.9972.2016111116240926010-601-8786-1yoyaru.dcm',
 '1.3.46.670589.11.78036.5.0.7904.2017021718234271002-601-7411-qgvsxs.dcm',
 '1.3.46.670589.11.78036.5.0.9972.2016111116240926010-601-1771-qwr0j1.dcm',
 '1.3.46.670589.11.78036.5.0.7904.2017021718234271002-601-376-p8bx85.dcm',
 '1.3.46.670589.11.78036.5.0.7904.2017021718234271002-601-11954-1xgq2n9.dcm',
 '1.3.46.670589.11.78036.5.0.9972.2016111116240926010-601-13488-ab7vex.dcm',
 '1.3.46.670589.11.78036.5.0.8324.2017022011243004006-601-8942-1wmqj0k.dcm',
 '1.3.46.670589.11.78036.5.0.9972.2016111116240926010-601-11133-8w49i.dcm',
 '1.3.46.670589.11.78036.5.0.7904.2017021718234271002-601-3792-1h2hjoo.dcm',
 '1.3.46.670589.11.78036.5.0.7904.2017021718234271002-601-9538-rdy0m1.dcm']

Matching files between servers for every sequence:

In [63]:
results = []
print set(upc_dict.keys()).difference(set(bsc_dict.keys())), 'are in UPC, missing in BSC'
print set(bsc_dict.keys()).difference(set(upc_dict.keys())), 'are in BSC, missing in UPC'
common_sq = set(bsc_dict.keys()).intersection(set(upc_dict.keys()))
print len(common_sq), 'in common'

for each in common_sq:
    row = []
    row.append(len(upc_dict[each]))
    row.append(len(bsc_dict[each]))
    row.append(len(set(upc_dict[each]).difference(set(bsc_dict[each]))))
    row.append(len(upc_dict[each]) - len(bsc_dict[each]))
    results.append(row)
pd.DataFrame(results, index=common_sq, columns=['UPCnet', 'BSC', 'Difference after matching', 'UPCnet - BSC (raw difference)'])

set([]) are in UPC, missing in BSC
set([]) are in BSC, missing in UPC
70 in common


,UPCnet,BSC,Difference after matching,UPCnet - BSC (raw difference)
604,135,135,0,0
601,4380660,4324409,56251,56251
1301,1268026,1249014,19012,19012
603,83,83,0,0
1303,90,90,0,0
704,3559,3559,0,0
1204,437,437,0,0
1201,47265,46997,268,268
701,196675,195771,904,904
1203,594,594,0,0
